In [17]:
import numpy as np
import pandas as pd
import time
import tpot
from datetime import datetime
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [52]:
print(f"Execution started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")

Execution started at 2022-02-20 18:33:51.


In [53]:
filepath = "../Data Preprocessing/iot23_combined_100k.csv"
df = pd.read_csv(filepath)

In [54]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,...,0,0,0,1,0,0,0,0,0,0
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444669,99994,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
1444670,99995,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
1444671,99996,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
1444672,99997,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0


In [55]:
del df['Unnamed: 0']

In [56]:
df['label'].value_counts()

PartOfAHorizontalPortScan     825939
Okiru                         262690
Benign                        197809
DDoS                          138777
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: label, dtype: int64

In [57]:
rank = 0
for label in df['label'].value_counts().index:
    if rank < 3:
        df.loc[(df.label == label), 'label'] = rank
    else:
        df.drop(df[df.label == label].index, inplace=True) 
    rank += 1
df = df.astype({'label': int})

In [58]:
df['label'].value_counts()

0    825939
1    262690
2    197809
Name: label, dtype: int64

In [59]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values

In [60]:
X.shape

(1286438, 24)

In [61]:
#Y = pd.get_dummies(df['label']).values
Y = df['label'].values

In [62]:
Y.shape

(1286438,)

In [63]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,2,0,...,0,0,0,1,0,0,0,0,0,0
4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,2,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444669,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
1444670,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
1444671,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
1444672,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [64]:
X = MinMaxScaler().fit_transform(X)

In [65]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV

estimator = DecisionTreeClassifier(random_state=10)
selector = RFECV(estimator, step=1, cv=5, n_jobs=-1, verbose=3)
selector = selector.fit(X, Y)
print('Support:',selector.support_)
print('Ranking:',selector.ranking_)
X = selector.transform(X)

Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Support: [ True False False False False False False False False  True False False
 False False False False False False False False False False False False]
Ranking: [ 1  5  6 20  3  8 14  7 19  1 21 11  9 13 12 16 1

In [66]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [67]:
X_train.shape

(1029150, 2)

In [68]:
Y_train

array([1, 0, 2, ..., 1, 0, 0])

In [ ]:
pipeline_optimizer = tpot.TPOTClassifier(generations=5,      #number of iterations to run the training
                                         population_size=10, #number of individuals to train
                                         verbosity=3,
                                         random_state=10)
pipeline_optimizer.fit(X_train, Y_train)                     #fit the pipeline optimizer - can take a long time
print(pipeline_optimizer.score(X_test, y_test))              #print scoring for the pipeline
pipeline_optimizer.export('tpot_exported_pipeline.py')       #export the pipeline - in Python code!

C:\Users\marce\Anaconda2\envs\doutorado\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(11.75,8.25))
ax.plot(np.arange(0,num_epochs), history.history['loss'], label='Loss', color='red')
ax.plot(np.arange(0,num_epochs), history.history['accuracy'], label='Accuracy', color='blue')
ax.grid(True, which='both')
ax.legend(loc='upper right')
ax.set_xlabel('Epoch')
ax.set_ylabel('Value')
ax.set_xlim(0, num_epochs)
ax.set_ylim(0, 1.1*np.max([np.max(history.history['loss']),np.max(history.history['accuracy'])]))
#ax.set_title(code.capitalize() + ' code ($M_{ep}=2^{' + str(nb_epoch_exp) + '})$ - ' + optimizer_name)

In [ ]:
print(f"Execution finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.")